In [2]:
# Librerias necesarias
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [3]:
# Cargar los datos
df = pd.read_excel("../data/0 TimeLog (original, no modificar).xlsx")

In [4]:
# Copio para no modificar el original
tl = df.copy()

In [5]:
# Agrego LOS a cada fila y reordeno
tl["LOS"] = (tl["TF"] - tl["TI"])
tl = tl[["ID", "MS_GRD", "UBICACIÓN", "TI", "TF", "LOS", "HOSPITAL", "UNIDAD"]]

# Lista para guardar filas nuevas
new_rows = []

# Recorrer filas consecutivas
for i in range(len(tl) - 1):
    row_current = tl.iloc[i]
    row_next = tl.iloc[i + 1]

    # Verificar condiciones
    same_id = row_current['ID'] == row_next['ID']
    same_ms = row_current['MS_GRD'] == row_next['MS_GRD']
    time_gap = row_current['TF'] < row_next['TI']
    same_hospital = row_current['HOSPITAL'] == row_next['HOSPITAL']

    if same_id and same_ms and time_gap:
        # Crear fila intermedia
        new_row = {
            'ID': row_current['ID'],
            'MS_GRD': row_current['MS_GRD'],
            'UBICACIÓN': f"{row_current['UBICACIÓN']} -> {row_next['UBICACIÓN']}",
            'TI': row_current['TF'],
            'TF': row_next['TI'],
            'LOS': row_next['TI'] - row_current['TF'],
            'HOSPITAL': row_current['HOSPITAL'],
            'UNIDAD': row_current['UNIDAD']
        }
        new_rows.append((i + 1, new_row))  # guardar con índice de inserción

# Lista para guardar todas las filas nuevas (dicts)
new_rows_list = [fila[1] for fila in new_rows]

# Convertir todo a un DataFrame una sola vez
new_rows_df = pd.DataFrame(new_rows_list)

# Concatenar en una sola pasada
tl = pd.concat([tl, new_rows_df], ignore_index=True)

# Reordenar filas para meter filas entremedio
tl = tl.sort_values(["ID", "TI"])

# Resetear index pra quie se vea ordenado
tl = tl.reset_index(drop=True)

In [6]:
# Ubicaciones
ubicaciones = list(tl["UBICACIÓN"].unique())
ubicaciones.sort()
ubicaciones

['Hospital_1_ED',
 'Hospital_1_GA',
 'Hospital_1_ICU',
 'Hospital_1_ICU -> Hospital_1_OR',
 'Hospital_1_ICU -> Hospital_1_SDU_WARD',
 'Hospital_1_OR',
 'Hospital_1_OR -> Hospital_1_ICU',
 'Hospital_1_OR -> Hospital_1_SDU_WARD',
 'Hospital_1_SDU_WARD',
 'Hospital_1_SDU_WARD -> Hospital_1_ICU',
 'Hospital_1_SDU_WARD -> Hospital_1_OR',
 'Hospital_2_ED',
 'Hospital_2_GA',
 'Hospital_2_ICU',
 'Hospital_2_ICU -> Hospital_2_OR',
 'Hospital_2_ICU -> Hospital_2_SDU_WARD',
 'Hospital_2_OR',
 'Hospital_2_OR -> Hospital_2_ICU',
 'Hospital_2_OR -> Hospital_2_SDU_WARD',
 'Hospital_2_SDU_WARD',
 'Hospital_2_SDU_WARD -> Hospital_2_ICU',
 'Hospital_2_SDU_WARD -> Hospital_2_OR',
 'Hospital_3_ED',
 'Hospital_3_GA',
 'Hospital_3_ICU',
 'Hospital_3_ICU -> Hospital_3_OR',
 'Hospital_3_ICU -> Hospital_3_SDU_WARD',
 'Hospital_3_OR',
 'Hospital_3_OR -> Hospital_3_ICU',
 'Hospital_3_OR -> Hospital_3_SDU_WARD',
 'Hospital_3_SDU_WARD',
 'Hospital_3_SDU_WARD -> Hospital_3_ICU',
 'Hospital_3_SDU_WARD -> Hospital_3_

In [7]:
tl.head(5)

,ID,MS_GRD,UBICACIÓN,TI,TF,LOS,HOSPITAL,UNIDAD
0,1,5,WL_WL,12,384,372,WL,WL
1,1,5,Hospital_1_GA,384,468,84,Hospital_1,GA
2,1,5,Hospital_1_ICU,468,552,84,Hospital_1,ICU
3,1,5,Hospital_1_ICU -> Hospital_1_SDU_WARD,552,576,24,Hospital_1,ICU
4,1,5,Hospital_1_SDU_WARD,576,768,192,Hospital_1,SDU_WARD


In [8]:
# tl.groupby('ID')['LOS'].sum().reset_index().describe()
#tl.head()
# tl.groupby('UBICACIÓN')['LOS'].mean().reset_index()
# tl.groupby('UBICACIÓN')['LOS'].mean().reset_index()["LOS"].tolist()

In [9]:
# Proporción de GRDs entre todos los casos (P_j)
variable = tl.groupby('ID')['MS_GRD'].max().reset_index()["MS_GRD"].value_counts(normalize=True).sort_index().reset_index()
P_j = {}
for index, row in variable.iterrows():
    P_j[int(row['MS_GRD'])] = row['proportion']

In [10]:
# LOS estando ya hospitalizado promedio por GRD (A_j)
# Solo considero cuando estan hospitalizados (se podria agregar GA facilmente)
#v1 = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD", "GA"])]
v1 = tl[tl["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]

v2 = v1.groupby('ID').agg({
    'LOS': 'sum',
    'MS_GRD': 'min'
}).reset_index().groupby("MS_GRD")["LOS"].mean().reset_index()

A_j = {}
for index, row in v2.iterrows():
    A_j[int(row['MS_GRD'])] = row['LOS']

A_j

{1: 367.77918090714246,
 2: 441.38515023590764,
 3: 183.812156942468,
 4: 319.45811965811964,
 5: 231.18476668759155,
 6: 158.58976533785693,
 7: 354.1263784351479,
 8: 308.61802187887673}

In [11]:
# LOS promedio por hospital_i y GRD_j (a_i_j)
a_i_j = {}
for numero in range(1,4):
    v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
    v2 = v1[v1["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
    v3 = v2.groupby('ID').agg({
        'LOS': 'sum',
        'MS_GRD': 'min'
    }).reset_index().groupby("MS_GRD")["LOS"].mean().reset_index()
    for index, row in v3.iterrows():
        a_i_j[(numero,int(row['MS_GRD']))] = row['LOS']

In [ ]:
# Proporción de GRDs en cada hospital (p_i_j)
p_i_j = {}

for numero in range(1,4):
    # Filtro por hospital
    v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
    # Filtro que este en alguna de esas unidades, cuando ya estan aceptados por el hospital
    v2 = v1[v1["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
    v3 = v2.groupby('ID')['MS_GRD'].max().reset_index()["MS_GRD"].value_counts(normalize=True).sort_index().reset_index()
    for index, row in v3.iterrows():
            p_i_j[(numero,int(row['MS_GRD']))] = row['proportion']

0.17535672861651336

307.748435745227

In [53]:
mean_los_i = {}
for numero in range(1,4):
    v1 = tl[tl["HOSPITAL"] == f"Hospital_{numero}"]
    v2 = v1[v1["UNIDAD"].isin(["ICU", "OR", "SDU_WARD"])]
    v3 = v2.groupby('ID').agg({
        'LOS': 'sum',
        'MS_GRD': 'min'
    }).reset_index()
    mean_los_i[numero] = v3["LOS"].mean()
print(mean_los_i)

# Case mix adjusted LOS
cmal_i = {}
for i in range(1,4):
    suma = 0
    for j in range(1, 9):
        suma += a_i_j[(i, j)] * P_j[j]
    cmal_i[i] = suma
print(cmal_i)

# LOS weighted case mix
lwcm_i = {}
for i in range(1,4):
    suma = 0
    for j in range(1, 9):
        suma += A_j[j] * p_i_j[(i, j)]
    lwcm_i[i] = suma
print(lwcm_i)

        

{1: 309.65457340921967, 2: 296.7760730640863, 3: 307.748435745227}
{1: 293.038531534336, 2: 290.42178975479305, 3: 289.8218741116188}
{1: 307.7236136122812, 2: 296.84952287794425, 3: 309.58670420333914}
